In [3]:
"""
MODIFIED: (efv) Use STSb-multi-mt Spanish
source: https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/sts/training_stsbenchmark.py

---

This examples trains BERT (or any other transformer model like RoBERTa, DistilBERT etc.) for the STSbenchmark from scratch. It generates sentence embeddings
that can be compared using cosine-similarity to measure the similarity.

Usage:
python training_nli.py

OR
python training_nli.py pretrained_transformer_model_name
"""
from torch.utils.data import DataLoader
from torch import cuda
import math
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import sys
import os
import gzip
import csv

from datasets import load_dataset

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

# logging.info(f"CUDA Device Name:{cuda.get_device_name()}")

MODEL = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

#You can specify any huggingface/transformers pre-trained model here, for example, bert-base-uncased, roberta-base, xlm-roberta-base
#model_name = sys.argv[1] if len(sys.argv) > 1 else 'distilbert-base-uncased'
model_name = MODEL 

# Read the dataset
train_batch_size = 32
num_epochs = 12
model_save_path = 'output/mine_embedding_model_'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for mapping tokens to embeddings
word_embedding_model = models.Transformer(model_name)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# Convert the dataset to a DataLoader ready for training
logging.info("Read stsb-multi-mt train dataset")

train_samples = []
dev_samples = []
test_samples = []

def samples_from_dataset(dataset):
    samples = [InputExample(texts=[e['sentence1'], e['sentence2']], label=e['similarity_score'] / 5) \
        for e in dataset] 
    return samples

train_samples = samples_from_dataset(load_dataset("stsb_multi_mt", name="ru", split="train"))
dev_samples = samples_from_dataset(load_dataset("stsb_multi_mt", name="ru", split="dev"))
test_samples = samples_from_dataset(load_dataset("stsb_multi_mt", name="ru", split="test"))

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

initial_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, write_csv=False)
initial_evaluator(model)


2023-05-08 13:30:13 - Use pytorch device: cpu
2023-05-08 13:30:13 - Read stsb-multi-mt train dataset


2023-05-08 13:30:14 - Found cached dataset stsb_multi_mt (C:/Users/maria/.cache/huggingface/datasets/stsb_multi_mt/ru/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9)


2023-05-08 13:30:16 - Found cached dataset stsb_multi_mt (C:/Users/maria/.cache/huggingface/datasets/stsb_multi_mt/ru/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9)


2023-05-08 13:30:18 - Found cached dataset stsb_multi_mt (C:/Users/maria/.cache/huggingface/datasets/stsb_multi_mt/ru/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9)
2023-05-08 13:30:18 - EmbeddingSimilarityEvaluator: Evaluating the model on  dataset:


2023-05-08 13:30:38 - Cosine-Similarity :	Pearson: 0.7867	Spearman: 0.7932
2023-05-08 13:30:38 - Manhattan-Distance:	Pearson: 0.7727	Spearman: 0.7725
2023-05-08 13:30:38 - Euclidean-Distance:	Pearson: 0.7725	Spearman: 0.7724
2023-05-08 13:30:38 - Dot-Product-Similarity:	Pearson: 0.6188	Spearman: 0.6078


0.7932018056116618

In [4]:

logging.info("Read stsb-multi-mt dev dataset")
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))


## Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)


##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

#model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='stsb-multi-mt-test')
test_evaluator(model, output_path=model_save_path)


2023-05-08 13:31:08 - Read stsb-multi-mt dev dataset
2023-05-08 13:31:08 - Warmup-steps: 144



Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 1/360 [00:02<16:28,  2.75s/it]

Iteration:   1%|          | 2/360 [00:05<15:37,  2.62s/it]

Iteration:   1%|          | 3/360 [00:07<15:06,  2.54s/it]

Iteration:   1%|          | 4/360 [00:09<14:00,  2.36s/it]

Iteration:   1%|▏         | 5/360 [00:11<13:18,  2.25s/it]

Iteration:   2%|▏         | 6/360 [00:13<12:59,  2.20s/it]

Iteration:   2%|▏         | 7/360 [00:15<12:42,  2.16s/it]

Iteration:   2%|▏         | 8/360 [00:17<12:32,  2.14s/it]

Iteration:   2%|▎         | 9/360 [00:19<12:49,  2.19s/it]

Iteration:   3%|▎         | 10/360 [00:21<12:40,  2.17s/it]

Iteration:   3%|▎         | 11/360 [00:24<12:48,  2.20s/it]

Iteration:   3%|▎         | 12/360 [00:26<12:36,  2.17s/it]

Iteration:   4%|▎         | 13/360 [00:28<12:35,  2.18s/it]

Iteration:   4%|▍         | 14/360 [00:30<12:40,  2.20s/it]

Iteration:   4%|▍         | 15/360 [00:33<12:36,  2.19s/

2023-05-08 13:43:57 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 0:


                                                                                 
                                                                                 
                                                                                 
                                                                                 
                                                                                 
Epoch:   0%|          | 0/4 [13:15<?, ?it/s]

2023-05-08 13:44:23 - Cosine-Similarity :	Pearson: 0.8610	Spearman: 0.8589
2023-05-08 13:44:23 - Manhattan-Distance:	Pearson: 0.8226	Spearman: 0.8286
2023-05-08 13:44:23 - Euclidean-Distance:	Pearson: 0.8228	Spearman: 0.8293
2023-05-08 13:44:23 - Dot-Product-Similarity:	Pearson: 0.7536	Spearman: 0.7786
2023-05-08 13:44:23 - Save model to output/mine_embedding_model_sentence-transformers-paraphrase-multilingual-MiniLM-L12-v2-2023-05-08_13-27-39


...

2023-05-08 14:24:07 - Cosine-Similarity :	Pearson: 0.8643	Spearman: 0.8630
2023-05-08 14:24:07 - Manhattan-Distance:	Pearson: 0.8215	Spearman: 0.8285
2023-05-08 14:24:07 - Euclidean-Distance:	Pearson: 0.8222	Spearman: 0.8296
2023-05-08 14:24:07 - Dot-Product-Similarity:	Pearson: 0.7561	Spearman: 0.7818
2023-05-08 14:24:07 - Save model to output/mine_embedding_model_sentence-transformers-paraphrase-multilingual-MiniLM-L12-v2-2023-05-08_13-27-39



Epoch: 100%|██████████| 4/4 [53:00<00:00, 795.02s/it]

2023-05-08 14:24:08 - EmbeddingSimilarityEvaluator: Evaluating the model on stsb-multi-mt-test dataset:


2023-05-08 14:24:29 - Cosine-Similarity :	Pearson: 0.8232	Spearman: 0.8214
2023-05-08 14:24:29 - Manhattan-Distance:	Pearson: 0.7980	Spearman: 0.7977
2023-05-08 14:24:29 - Euclidean-Distance:	Pearson: 0.7978	Spearman: 0.7981
2023-05-08 14:24:29 - Dot-Product-Similarity:	Pearson: 0.7037	Spearman: 0.7115


0.821449405018268